# <div align="center" style="color: #ff5733;">New Pos Addition and Mapping</div>

In [1]:
# %% [markdown]
# # Jupyter Notebook Loading Header
#
# This is a custom loading header for Jupyter Notebooks in Visual Studio Code.
# It includes common imports and settings to get you started quickly.

# %% [markdown]
## Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.cloud import bigquery
import os
path = r'C:\Users\DwaipayanChakroborti\AppData\Roaming\gcloud\legacy_credentials\dchakroborti@tonikbank.com\adc.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = path
client = bigquery.Client(project='prj-prod-dataplatform')

# %% [markdown]
## Configure Settings
# Set options or configurations as needed
# Example: pd.set_option('display.max_columns', None)

C:\Users\DwaipayanChakroborti\AppData\Roaming\Python\Python310\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv(r"C:\Users\DwaipayanChakroborti\OneDrive - Tonik Financial Pte Ltd\MyStuff\SIL Sales\Shin Nebria\POS_Addition\Data\List of Store Opening_20240415.csv")
df

,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM,Brand
0,Ansons Alabang,Alabang Town Center,For Opening,15-Apr-24,M67TDGB,NCR South 2,Grace Belarmino,Ansons
1,Ansons BGC,Standalone,For Opening,15-Apr-24,MHNT3WR,NCR South 1,Klein Justine Tamayo,Ansons
2,Ansons Cainta,Q-Plaza Cainta,For Opening,15-Apr-24,MDTBKRI,NCR East,Albee David,Ansons
3,Ansons Capitol Commons,Capitol Commons,For Opening,15-Apr-24,M9EX1KL,NCR East,Albee David,Ansons
4,Ansons Cash and Carry,Cash & Carry,For Opening,15-Apr-24,M3ERUVC,NCR South 1,Klein Justine Tamayo,Ansons
...,...,...,...,...,...,...,...,...
180,Xtreme Appliances Laoag,Standalone,For Opening,14-Apr-24,MCJ3BVQ,North Luzon 2,Rachelle Belmonte,Xtreme
181,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,14-Apr-24,MO3AHGW,NCR East,Albee David,Xtreme
182,Xtreme Appliances Metro Plaza,Standalone,For Opening,14-Apr-24,M8WN4DK,NCR North,Sander Maghirang,Xtreme
183,Xtreme appliances Robinsons Malabon,Standalone,For Opening,14-Apr-24,M6T5UNY,NCR North,Sander Maghirang,Xtreme


In [3]:
df.rename(columns = {'Brand': 'BrandStores'}, inplace = True)
df.columns

Index(['POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY',
       'REGION', 'TSM', 'BrandStores'],
      dtype='object')

In [4]:
df['BrandStores'].value_counts()

BrandStores
Rulls         124
MemoXpress     37
Ansons         15
Xtreme          9
Name: count, dtype: int64

In [5]:
df['PURPLE KEY'].value_counts()

PURPLE KEY
M67TDGB    1
MWZPTIY    1
MA0C1DZ    1
MQVP6H8    1
MQV6OR3    1
          ..
MOVZDHU    1
MUKYPLB    1
M80SH64    1
MN9OHTI    1
M8GKOQ6    1
Name: count, Length: 185, dtype: int64

In [6]:
df = df[['BrandStores', 'POS NAME', 'STORE LOCATION', 'STATUS', 'DATE ACTIVATION', 'PURPLE KEY', 'REGION', 'TSM']]
df

,BrandStores,POS NAME,STORE LOCATION,STATUS,DATE ACTIVATION,PURPLE KEY,REGION,TSM
0,Ansons,Ansons Alabang,Alabang Town Center,For Opening,15-Apr-24,M67TDGB,NCR South 2,Grace Belarmino
1,Ansons,Ansons BGC,Standalone,For Opening,15-Apr-24,MHNT3WR,NCR South 1,Klein Justine Tamayo
2,Ansons,Ansons Cainta,Q-Plaza Cainta,For Opening,15-Apr-24,MDTBKRI,NCR East,Albee David
3,Ansons,Ansons Capitol Commons,Capitol Commons,For Opening,15-Apr-24,M9EX1KL,NCR East,Albee David
4,Ansons,Ansons Cash and Carry,Cash & Carry,For Opening,15-Apr-24,M3ERUVC,NCR South 1,Klein Justine Tamayo
...,...,...,...,...,...,...,...,...
180,Xtreme,Xtreme Appliances Laoag,Standalone,For Opening,14-Apr-24,MCJ3BVQ,North Luzon 2,Rachelle Belmonte
181,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,14-Apr-24,MO3AHGW,NCR East,Albee David
182,Xtreme,Xtreme Appliances Metro Plaza,Standalone,For Opening,14-Apr-24,M8WN4DK,NCR North,Sander Maghirang
183,Xtreme,Xtreme appliances Robinsons Malabon,Standalone,For Opening,14-Apr-24,M6T5UNY,NCR North,Sander Maghirang


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   BrandStores      185 non-null    object
 1   POS NAME         185 non-null    object
 2   STORE LOCATION   185 non-null    object
 3   STATUS           185 non-null    object
 4   DATE ACTIVATION  185 non-null    object
 5   PURPLE KEY       185 non-null    object
 6   REGION           185 non-null    object
 7   TSM              185 non-null    object
dtypes: object(8)
memory usage: 11.7+ KB


In [8]:
df.rename(columns={'POS NAME':'POS', 'DATE ACTIVATION':'DateOpened', 'STORE LOCATION': 'StoreLocation', 'STATUS':'Status', 'PURPLE KEY':'PurpleKey', 'REGION':'Region'}, inplace = True)

In [10]:
import pandas as pd

df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%d-%b-%y')

In [11]:
print(df['DateOpened'].dtype)
# datetime64[ns]

datetime64[ns]


In [12]:
print(df['DateOpened'].iloc[0]) 
# 2024-01-15 00:00:00

2024-04-15 00:00:00


In [13]:
df['DateOpened'] = df['DateOpened'].dt.strftime('%Y-%m-%d')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BrandStores    185 non-null    object
 1   POS            185 non-null    object
 2   StoreLocation  185 non-null    object
 3   Status         185 non-null    object
 4   DateOpened     185 non-null    object
 5   PurpleKey      185 non-null    object
 6   Region         185 non-null    object
 7   TSM            185 non-null    object
dtypes: object(8)
memory usage: 11.7+ KB


In [14]:
df['DateOpened'] = pd.to_datetime(df['DateOpened'], format='%Y-%m-%d')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 185 entries, 0 to 184
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   BrandStores    185 non-null    object        
 1   POS            185 non-null    object        
 2   StoreLocation  185 non-null    object        
 3   Status         185 non-null    object        
 4   DateOpened     185 non-null    datetime64[ns]
 5   PurpleKey      185 non-null    object        
 6   Region         185 non-null    object        
 7   TSM            185 non-null    object        
dtypes: datetime64[ns](1), object(7)
memory usage: 11.7+ KB


In [16]:
df.head()

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,Ansons,Ansons Alabang,Alabang Town Center,For Opening,2024-04-15,M67TDGB,NCR South 2,Grace Belarmino
1,Ansons,Ansons BGC,Standalone,For Opening,2024-04-15,MHNT3WR,NCR South 1,Klein Justine Tamayo
2,Ansons,Ansons Cainta,Q-Plaza Cainta,For Opening,2024-04-15,MDTBKRI,NCR East,Albee David
3,Ansons,Ansons Capitol Commons,Capitol Commons,For Opening,2024-04-15,M9EX1KL,NCR East,Albee David
4,Ansons,Ansons Cash and Carry,Cash & Carry,For Opening,2024-04-15,M3ERUVC,NCR South 1,Klein Justine Tamayo


In [17]:
sq = """drop table if exists prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
try:
    d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
    print("Table still exists")
except:
    print("Table dropped")

Query is running:   0%|          |Table dropped


In [18]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1
as 
select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;
"""
client.query(sq)



QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=4547bfc0-a4b5-4dd8-b01f-8acb837318f4>

In [19]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
d = client.query(sq).to_dataframe(progress_bar_type='tqdm')
print(f"The shape of d is:\t{d.shape}")

Job ID e08dada8-ec35-4bb0-b4fc-f8e039b82b40 successfully executed: 100%|██████████|
Query is running:   0%|          |
Downloading: 100%|██████████|
The shape of d is:	(327, 8)


In [20]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""

dfsil = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')

Job ID 17c34ef6-290d-4d66-bbee-00949b64ca82 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


In [21]:
dfsil.BrandStores.value_counts()

BrandStores
FC Home             108
Home Along           90
ESLA                 30
Homeworks            13
Xtreme               13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
Sogo Home             3
Willy & Sons          3
A2RJ                  3
Hardware Sugar        2
Affordables           2
KUM Home Center       2
PHPS                  2
Junliz                2
Home Factory          2
Dizon Furniture       1
JTG                   1
KUM Home              1
Name: count, dtype: int64

In [22]:
dfsil.shape

(327, 8)

In [23]:
# Set table_id to the ID of the table to append to.
table_id = "prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"

job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_APPEND"
)

job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  

job.result()  # Wait for the job to complete.

table = client.get_table(table_id)  # Make an API request.
print(
    "Loaded {} rows and {} columns to {}".format(
        table.num_rows, len(table.schema), table_id
    )
)

Loaded 512 rows and 8 columns to prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1


In [24]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1"""
df = client.query(sq).to_dataframe(progress_bar_type = 'tqdm')
print(f"The shape of the dataframe is: {df.shape}")

Job ID ab74f4f5-b6b4-4d8a-98f1-515999ed2f91 successfully executed: 100%|██████████|
Downloading: 100%|██████████|
The shape of the dataframe is: (512, 8)


In [25]:
df.tail(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
502,FC Home,FC Home Center Mamatid,Standalone,POS Operational,2023-09-14,MC31AV7,South Luzon 2,Christian Klide Alberto
503,FC Home,FC Home Center Binan,Central Mall Laguna,POS Operational,2023-09-14,MYRK3EJ,South Luzon 2,Christian Klide Alberto
504,FC Home,FC Home Center Sta Cruz,Standalone,POS Operational,2023-09-14,M1ZYHKU,South Luzon 2,Christian Klide Alberto
505,FC Home,FC Home Center Cabuyao 1,Standalone,POS Operational,2023-09-14,M5D8JOH,South Luzon 2,Christian Klide Alberto
506,FC Home,FC Home Center San Pablo,Standalone,POS Operational,2023-09-14,MADP0JZ,South Luzon 2,Christian Klide Alberto
507,FC Home,FC Home Center Siniloan,Standalone,POS Operational,2023-09-15,MANB6XQ,South Luzon 2,Christian Klide Alberto
508,Workstation,Workstation Home Appliances Central Mall,Central Mall,POS Operational,2023-10-13,MTNCOR5,South Luzon 2,Christian Klide Alberto
509,Workstation,Workstation Home Appliances Pavilion Mall 1,Pavillon Mall,POS Operational,2023-10-13,MN8MWPJ,South Luzon 2,Christian Klide Alberto
510,Workstation,Workstation Home Appliances Target Mall,Target Mall,POS Operational,2023-10-13,M8I4QGT,South Luzon 2,Christian Klide Alberto
511,Xtreme,Xtreme Appliances Calamba,Carmel Mall,POS Operational,2023-10-15,MUBPW04,South Luzon 2,Christian Klide Alberto


In [26]:
df['BrandStores'].value_counts()

BrandStores
Rulls               124
FC Home             108
Home Along           90
MemoXpress           37
ESLA                 30
Xtreme               22
Ansons               15
Homeworks            13
Automatic Centre     11
Kservico             10
Faith Glorious        9
Jhunpyo               8
Workstation           6
Power Mac Center      5
A2RJ                  3
Sogo Home             3
Willy & Sons          3
Hardware Sugar        2
Affordables           2
KUM Home Center       2
PHPS                  2
Junliz                2
Home Factory          2
JTG                   1
Dizon Furniture       1
KUM Home              1
Name: count, dtype: int64

In [27]:
# Find duplicate values in 'POS' column
duplicates = df[df.duplicated('POS', keep=False)]

# Print the entire rows for duplicate entries in 'POS'
if not duplicates.empty:
    print("Duplicate values in 'POS' column:")
    duplicates.to_csv("Temp.csv")
else:
    print("No duplicate values found in 'POS' column.")
    
dd = pd.read_csv("Temp.csv")
dd.sort_values(by='POS')

Duplicate values in 'POS' column:


,Unnamed: 0,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,186,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,MV4UO0X,NaN,NaN
1,187,Hardware Sugar,Hardware Sugar Makati,NaN,NaN,2022-07-04,M9ZK6IU,NaN,NaN
12,474,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MGP2AQI,NCR North,Ben Antonio Lacsamana
13,476,Home Along,Home Along - Mayon,Standalone,POS Operational,2022-07-01,MN3GL66,NCR North 2,Ben Antonio Lacsamana
14,481,Home Along,Home Along Malanday,Standalone,POS Operational,2022-07-11,MNBW4RZ,NCR North,Ben Antonio Lacsamana
15,482,Home Along,Home Along Malanday,NaN,NaN,2022-07-11,MURFAIV,NCR North,Ben Antonio Lacsamana
4,396,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MDGYHQJ,NCR South,Klein Justine Tamayo
5,398,Home Along,Home Along Muntinlupa,Standalone,POS Operational,2022-03-26,MU82X9Z,NCR South,Klein Justine Tamayo
8,410,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MRGP0WQ,NCR South,Klein Justine Tamayo
11,413,Home Along,Home Along Puregold Taguig,Standalone,POS Operational,2022-07-01,MDO3KI4,NCR South,Klein Justine Tamayo


In [28]:
pd.set_option('display.max_rows', None)
# Count the occurrences of each value in the 'PurpleKey' column
purple_key_counts = df['PurpleKey'].value_counts()

# Filter to include only values with count >= 2
filtered_counts = purple_key_counts[purple_key_counts >= 2]

# Display the filtered counts
print(filtered_counts)


Series([], Name: count, dtype: int64)


In [29]:
df[df['PurpleKey']== 'M4EYWQX']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
92,Ansons,Ansons Landmark Filinvest,Landmark Filinvest,For Opening,2024-04-15,M4EYWQX,NCR South 2,Grace Belarmino


In [30]:
sq = """drop table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=9e9aa2e6-d007-4dd9-ad88-805f4135112f>

In [31]:
sq = """create table prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING as select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING1;"""
client.query(sq)


QueryJob<project=prj-prod-dataplatform, location=asia-southeast1, id=67ce06db-bad6-4254-b6a2-95719092d03b>

In [32]:
client.query("commit")

QueryJob<project=prj-prod-dataplatform, location=US, id=039fdcb8-f642-4894-b6a6-2ac89b9dd6c3>

In [33]:
sq = """select * from prj-prod-dataplatform.dap_ds_poweruser_playground.POS_TSM_MAPIING;"""

df = client.query(sq).to_dataframe(progress_bar_type='tqdm')
df.shape

Job ID 58322785-991f-43a6-8eec-119a80cf4738 successfully executed: 100%|██████████|
Downloading: 100%|██████████|


(512, 8)

In [35]:
df[df['BrandStores']=='Rulls']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
7,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
8,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David
10,Rulls,Mobilecraze K2 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MUFDX4A,NCR East,Albee David
12,Rulls,Mobile Craze PG Tanay Rizal,Puregold Tanay Rizal,For Opening,2024-04-15,MFCWRJQ,NCR East,Albee David
14,Rulls,Mobilecraze K1 PG Marikina,Puregold Marikina,For Opening,2024-04-15,M9KLTD1,NCR East,Albee David
15,Rulls,Mobile Craze Town Center Mall Tanay,Craze Town Center Mall Tanay,For Opening,2024-04-15,M3ED6SF,NCR East,Albee David
17,Rulls,MBC Montalban Town Center,Montalban Town Center,For Opening,2024-04-15,MI9KLNA,NCR East,Albee David
18,Rulls,MBC Gaisano Inline Binangonan,Gaisano Inline Binangonan,For Opening,2024-04-15,M38S4QA,NCR East,Albee David
21,Rulls,Mobilecraze K3 PG Marikina,Puregold Marikina,For Opening,2024-04-15,MW6HOE7,NCR East,Albee David
22,Rulls,Audionet Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,M8BPO3Y,NCR East,Albee David


In [36]:
df.head(10)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
0,FC Home,FC Home Center Balibago,LOT 5 BLK 9 GARNET ST. RHINELAND COMMERCIAL CO...,Operational,2023-08-01,MOG243Y,South Luzon 2,Klide Alberto
1,Xtreme,Xtreme Appliances Anabu,None,None,2022-08-27,MR2YDE8,None,None
2,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,M9ZK6IU,None,None
3,Hardware Sugar,Hardware Sugar Makati,None,None,2022-07-04,MV4UO0X,None,None
4,Home Along,Home Along Sumulong,None,None,2022-07-12,M2G78K4,None,None
5,Dizon Furniture,Dizon Furniture Tiendesitas,None,None,2022-07-23,MOE3JQH,None,None
6,Xtreme,Xtreme Appliances Lucky Gold Plaza,Lucky Gold Plaza,For Opening,2024-04-14,MO3AHGW,NCR East,Albee David
7,Rulls,Realme Robinsons Metro East,Robinsons Metro East,For Opening,2024-04-15,MBLR71F,NCR East,Albee David
8,Rulls,Rulls Antipolo,Robinsons Antipolo,For Opening,2024-04-15,M8U05KZ,NCR East,Albee David
9,Ansons,Ansons Cainta,Q-Plaza Cainta,For Opening,2024-04-15,MDTBKRI,NCR East,Albee David


In [37]:
df.sort_values('DateOpened', ascending=False)

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
194,Rulls,Tecno Market Mall Paco,Market Mall Paco,For Opening,2024-04-15,M3PEIJ0,NCR North,Sander Maghirang
212,MemoXpress,Mx Robinsons Place Lipa,Robinsons Lipa,For Opening,2024-04-15,M4HZOK3,South Luzon 3,Jan jan Dela Cruz
189,MemoXpress,Memoxpress Trinoma,Trinoma,For Opening,2024-04-15,MFAOPRK,NCR North,Sander Maghirang
190,Rulls,Audionet Robinson Novaliches,Robinsons Novaliches,For Opening,2024-04-15,MHB8C7A,NCR North,Sander Maghirang
191,Rulls,Stormtech Alimall Cubao,Alimall Cubao,For Opening,2024-04-15,MJ81NOS,NCR North,Sander Maghirang
192,Ansons,Ansons Landmark Trinoma,Trinoma,For Opening,2024-04-15,M7RP30A,NCR North,Sander Maghirang
193,Rulls,Mobilecraze Market Mall Paco,Market Mall Paco,For Opening,2024-04-15,MY9Z0PA,NCR North,Sander Maghirang
388,MemoXpress,Memoxpress Market Market Taguig,Market Market,For Opening,2024-04-15,MN9OHTI,NCR South 1,Klein Justine Tamayo
195,Ansons,Ansons Trinoma,Trinoma,For Opening,2024-04-15,MP1IOL2,NCR North,Sander Maghirang
196,MemoXpress,Memoxpress Robinsons Place Midtown,Robinsons Midtown,For Opening,2024-04-15,MUJ3NYX,NCR North,Sander Maghirang


In [38]:
df[df['PurpleKey'] =='M3Z10RP']

,BrandStores,POS,StoreLocation,Status,DateOpened,PurpleKey,Region,TSM
362,Ansons,Ansons Greenhills,Greenhills,For Opening,2024-04-15,M3Z10RP,NCR South 1,Klein Justine Tamayo
